**Limpieza de datos Base Siniestralidad 2020 - 2024**

In [1]:
#instalacion e importacion de librerias necesarias

%pip install geopandas

import pandas as pd
import re
import geopandas as gpd
import numpy as np
import os
import unicodedata
import time
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
#from google.colab import drive # Descomenta si usas Google Colab






Note: you may need to restart the kernel to use updated packages.


In [2]:
# Cargar el archivo Excel
excel_path = "../data/raw/base_anuario_de_siniestralidad_2024_Final.xlsx"
archivo_excel = pd.ExcelFile(excel_path)

# Ver las hojas disponibles
print("Hojas disponibles en el archivo:")
print(archivo_excel.sheet_names)

Hojas disponibles en el archivo:
['Siniestros', 'Vehiculos', 'Actor_vial', 'Hoja1', 'Limpieza de datos', 'Diccionario']


In [3]:
#cargar la hoja 'Siniestros' es mi hoja de interes segun el analisis previo y la asignacion de tareas,segmentamos la data solo por los años de 2020 a 2024, generamos 2 dataframes
df_siniestros = pd.read_excel(excel_path, sheet_name='Siniestros')


df_siniestros = df_siniestros[(df_siniestros['AA_Acc'] >= 2020) & (df_siniestros['AA_Acc'] <= 2024)]


# Verificar resultado, tamaño del df resultante
print(df_siniestros['AA_Acc'].unique())
print(df_siniestros.shape)


[2020 2021 2022 2023 2024]
(105094, 40)


In [5]:
df_1 = df_siniestros.copy()
df_2 = df_siniestros.copy()

**Transformacion df_1**

In [6]:
#validacion de contenido
df_1.head(10)

,Codigo_Accidente,Formulario,Longitud,Latitud,Direccion,Fecha_Acc,AA_Acc,MM_Acc,DD_Mes_Acc,Dia_Semana_Acc,...,Con_Sitp,Con_Troncal,Con_Alimentador,Con_Zonal,Con_Provisional,Con_Articulado,Con_Biarticulado,Con_Padron_Dual,Con_Servicio_Especial,Con_Taxi
169947,10509738,A001132426,-74.125084,4.672291,KR 86-CL 24 02,2020-01-31,2020,Enero,31,viernes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
169948,10509739,A001132129,-74.051785,4.732306,CL 150-KR 48 12,2020-01-31,2020,Enero,31,viernes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
169949,10509740,A001132599,-74.125649,4.672324,CL 24-KR 86 02,2020-01-31,2020,Enero,31,viernes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
169950,10509741,A001131825,-74.135092,4.542815,AV AVENIDA BOYACA-CL 15D S 02,2020-01-31,2020,Enero,31,viernes,...,SI,NaN,NaN,SI,NaN,NaN,NaN,NaN,NaN,NaN
169951,10509742,A001132476,-74.100031,4.561922,KR 4B-CL 36 S 53,2020-01-31,2020,Enero,31,viernes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
169952,10509743,A001130759,-74.096272,4.757223,KR 111-CL 153 30,2020-01-31,2020,Enero,31,viernes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
169953,10509744,A001130320,-74.180517,4.598093,CL 65-KR 77H S 2,2020-01-31,2020,Enero,31,viernes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
169954,10509745,A001131942,-74.154019,4.642562,AV AVENIDA CIUDAD DE CALI-CL 6C 02,2020-01-31,2020,Enero,31,viernes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
169955,10509746,A001131910,-74.123855,4.740633,CL 130A-KR 154 2,2020-01-31,2020,Enero,31,viernes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SI
169956,10509747,A001132237,-74.028708,4.704716,KR 7-CL 127B 02,2020-01-31,2020,Enero,31,viernes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SI


In [7]:
#conocer nombres de columnas, para renombramiento
print(df_siniestros.columns.tolist())

['Codigo_Accidente', 'Formulario', 'Longitud', 'Latitud', 'Direccion', 'Fecha_Acc', 'AA_Acc', 'MM_Acc', 'DD_Mes_Acc', 'Dia_Semana_Acc', 'Hora_Acc', 'Min_Acc', 'Localidad', 'Clase_Acc', 'Elemento_Choque', 'Tipo_Objeto_Fijo', 'Gravedad_Indicador_Tradicional', 'Gravedad_indicador_30d', 'Con_Bicicleta', 'Con_Carga', 'Con_Embriaguez', 'Con_Huecos', 'Con_Menores', 'Con_Moto', 'Con_Peaton', 'Con_Persona_Mayor', 'Con_Rutas', 'Con_Tpi', 'Con_Tpp', 'Con_Velocidad', 'Con_Sitp', 'Con_Troncal', 'Con_Alimentador', 'Con_Zonal', 'Con_Provisional', 'Con_Articulado', 'Con_Biarticulado', 'Con_Padron_Dual', 'Con_Servicio_Especial', 'Con_Taxi']


In [8]:

# 1. Diccionario general de columnas a renombrar
variables = {
    'Con_Embriaguez': 'Embriaguez',
    'Con_Huecos': 'Huecos',
    'Con_Menores': 'Persona_Menor',
    'Con_Peaton': 'Peaton',
    'Con_Persona_Mayor': 'Persona_Mayor',
    'Con_Rutas': 'Rutas',
    'Con_Tpi': 'Tpi',
    'Con_Tpp': 'Tpp',
    'Con_Velocidad': 'Velocidad',
    'Con_Sitp': 'Sitp',
    'Con_Troncal': 'Troncal'
}

# 2. Generar nuevo dataframe con las columnas renombradas
df = df_1.rename(columns=variables)



# 4. Factores ya renombrados
factores = list(variables.values())

# 5. Pasar a formato largo
df_largo = df.melt(
    id_vars=['Codigo_Accidente'],
    value_vars=factores,
    var_name='Factor',
    value_name='Valor'
)

# 6. Filtrar solo los que tienen 'SI'
df_largo = df_largo[df_largo['Valor'] == 'SI']

# 7. Crear DataFrames separados

# 7.1 Factores de Riesgo
factores_riesgo = ['Embriaguez', 'Huecos', 'Peaton',
                   'Persona_Menor', 'Persona_Mayor', 'Velocidad']

df_riesgo = df_largo[df_largo['Factor'].isin(factores_riesgo)].copy()
df_riesgo = df_riesgo.rename(columns={'Factor': 'Factores_de_Riesgo'})
df_riesgo = df_riesgo[['Codigo_Accidente', 'Factores_de_Riesgo']]

# 7.2 Tipo de Transporte
tipos_transporte = ['Rutas', 'Tpi', 'Tpp', 'Sitp', 'Troncal']

df_transporte = df_largo[df_largo['Factor'].isin(tipos_transporte)].copy()
df_transporte = df_transporte.rename(columns={'Factor': 'Tipo_de_Transporte'})
df_transporte = df_transporte[['Codigo_Accidente', 'Tipo_de_Transporte']]

**Transformacion y limpieza df_2**

In [9]:
#verificar las filas y columnas disponibles
df_2.shape

(105094, 40)

In [10]:
# definir nuestras columnas de interes, para posterormente segmentar la data y tomar los registros pertenecientes a 2020 a 2024
print("Columnas disponibles:")
for i, col in enumerate(df_2.columns, 1):
    print(f"{i}. {col}")

Columnas disponibles:
1. Codigo_Accidente
2. Formulario
3. Longitud
4. Latitud
5. Direccion
6. Fecha_Acc
7. AA_Acc
8. MM_Acc
9. DD_Mes_Acc
10. Dia_Semana_Acc
11. Hora_Acc
12. Min_Acc
13. Localidad
14. Clase_Acc
15. Elemento_Choque
16. Tipo_Objeto_Fijo
17. Gravedad_Indicador_Tradicional
18. Gravedad_indicador_30d
19. Con_Bicicleta
20. Con_Carga
21. Con_Embriaguez
22. Con_Huecos
23. Con_Menores
24. Con_Moto
25. Con_Peaton
26. Con_Persona_Mayor
27. Con_Rutas
28. Con_Tpi
29. Con_Tpp
30. Con_Velocidad
31. Con_Sitp
32. Con_Troncal
33. Con_Alimentador
34. Con_Zonal
35. Con_Provisional
36. Con_Articulado
37. Con_Biarticulado
38. Con_Padron_Dual
39. Con_Servicio_Especial
40. Con_Taxi


In [14]:
# Identificar columnas que contienen "con_"
cols_a_eliminar = [col for col in df_2.columns if "con_" in col.lower()]

print("Columnas eliminadas:")
print(cols_a_eliminar)

# Eliminar las columnas
df_2 = df_2.drop(columns=cols_a_eliminar)

# Confirmar columnas restantes
print("\nColumnas restantes en el DataFrame:")
print(df_2.columns.tolist())

Columnas eliminadas:
['Con_Bicicleta', 'Con_Carga', 'Con_Embriaguez', 'Con_Huecos', 'Con_Menores', 'Con_Moto', 'Con_Peaton', 'Con_Persona_Mayor', 'Con_Rutas', 'Con_Tpi', 'Con_Tpp', 'Con_Velocidad', 'Con_Sitp', 'Con_Troncal', 'Con_Alimentador', 'Con_Zonal', 'Con_Provisional', 'Con_Articulado', 'Con_Biarticulado', 'Con_Padron_Dual', 'Con_Servicio_Especial', 'Con_Taxi']

Columnas restantes en el DataFrame:
['Codigo_Accidente', 'Formulario', 'Longitud', 'Latitud', 'Direccion', 'Fecha_Acc', 'AA_Acc', 'MM_Acc', 'DD_Mes_Acc', 'Dia_Semana_Acc', 'Hora_Acc', 'Min_Acc', 'Localidad', 'Clase_Acc', 'Elemento_Choque', 'Tipo_Objeto_Fijo', 'Gravedad_Indicador_Tradicional', 'Gravedad_indicador_30d']


In [15]:
# Verificar columnas restantes
for col in df_2.columns:
    print(col)

Codigo_Accidente
Formulario
Longitud
Latitud
Direccion
Fecha_Acc
AA_Acc
MM_Acc
DD_Mes_Acc
Dia_Semana_Acc
Hora_Acc
Min_Acc
Localidad
Clase_Acc
Elemento_Choque
Tipo_Objeto_Fijo
Gravedad_Indicador_Tradicional
Gravedad_indicador_30d


In [16]:
#eliminar columnas irrelevantes que no sirven para el analisis y verificamos que se hayan eliminado correctamente

df_2 = df_2.drop(columns=['Formulario'])

# Verificar resultado
df_2.head()

,Codigo_Accidente,Longitud,Latitud,Direccion,Fecha_Acc,AA_Acc,MM_Acc,DD_Mes_Acc,Dia_Semana_Acc,Hora_Acc,Min_Acc,Localidad,Clase_Acc,Elemento_Choque,Tipo_Objeto_Fijo,Gravedad_Indicador_Tradicional,Gravedad_indicador_30d
169947,10509738,-74.125084,4.672291,KR 86-CL 24 02,2020-01-31,2020,Enero,31,viernes,10,41,FONTIBÓN,Choque,Vehículo,NaN,Solo Daños,Solo Daños
169948,10509739,-74.051785,4.732306,CL 150-KR 48 12,2020-01-31,2020,Enero,31,viernes,21,23,SUBA,Choque,Vehículo,NaN,Solo Daños,Solo Daños
169949,10509740,-74.125649,4.672324,CL 24-KR 86 02,2020-01-31,2020,Enero,31,viernes,10,20,FONTIBÓN,Choque,Vehículo,NaN,Solo Daños,Solo Daños
169950,10509741,-74.135092,4.542815,AV AVENIDA BOYACA-CL 15D S 02,2020-01-31,2020,Enero,31,viernes,12,18,CIUDAD BOLÍVAR,Choque,Vehículo,NaN,Solo Daños,Solo Daños
169951,10509742,-74.100031,4.561922,KR 4B-CL 36 S 53,2020-01-31,2020,Enero,31,viernes,1,12,SAN CRISTÓBAL,Choque,Objeto Fijo,VEHICULO,Solo Daños,Solo Daños


In [17]:
# renombramiento de columnas para un mejor entendimiento de las mismas, validacion
df_2 = df_2.rename(columns={
    "AA_Acc": "Año_Accidente",
    "Fecha_Acc": "Fecha_Accidente",
    "MM_Acc": "Mes_Accidente",
    "DD_Acc": "Día_Accidente",
    "DD_Mes_Acc": "Día_Accidente",
    "Dia_Semana_Acc": "Día_Semana_Accidente",
    "Hora_Acc": "Hora_Accidente",
    "Min_Acc": "Minuto_Accidente",
    "Clase_Acc": "Clase_Accidente",
})
df_2.head()

,Codigo_Accidente,Longitud,Latitud,Direccion,Fecha_Accidente,Año_Accidente,Mes_Accidente,Día_Accidente,Día_Semana_Accidente,Hora_Accidente,Minuto_Accidente,Localidad,Clase_Accidente,Elemento_Choque,Tipo_Objeto_Fijo,Gravedad_Indicador_Tradicional,Gravedad_indicador_30d
169947,10509738,-74.125084,4.672291,KR 86-CL 24 02,2020-01-31,2020,Enero,31,viernes,10,41,FONTIBÓN,Choque,Vehículo,NaN,Solo Daños,Solo Daños
169948,10509739,-74.051785,4.732306,CL 150-KR 48 12,2020-01-31,2020,Enero,31,viernes,21,23,SUBA,Choque,Vehículo,NaN,Solo Daños,Solo Daños
169949,10509740,-74.125649,4.672324,CL 24-KR 86 02,2020-01-31,2020,Enero,31,viernes,10,20,FONTIBÓN,Choque,Vehículo,NaN,Solo Daños,Solo Daños
169950,10509741,-74.135092,4.542815,AV AVENIDA BOYACA-CL 15D S 02,2020-01-31,2020,Enero,31,viernes,12,18,CIUDAD BOLÍVAR,Choque,Vehículo,NaN,Solo Daños,Solo Daños
169951,10509742,-74.100031,4.561922,KR 4B-CL 36 S 53,2020-01-31,2020,Enero,31,viernes,1,12,SAN CRISTÓBAL,Choque,Objeto Fijo,VEHICULO,Solo Daños,Solo Daños


In [18]:
#normalizar los registros de la columna Tipo_Objeto_Fijo, validacion de cambios
df_2['Tipo_Objeto_Fijo'] = df_2['Tipo_Objeto_Fijo'].str.capitalize()
df_2['Tipo_Objeto_Fijo'].unique()

array([nan, 'Vehiculo', 'Tren', 'Arbol', 'Semaforo', 'Objeto fijo',
       'Poste', 'Inmueble', 'Muro', 'Semoviente', 'Por identificar',
       'Baranda', 'Vehiculo estacionado', 'Valla-señal', 'Otro',
       'Tarima-caseta', 'Hidrante'], dtype=object)

In [19]:
#imputacions de datos faltantes en la columna Tipo_Objeto_Fijo con el valor "No aplica"
df_2['Tipo_Objeto_Fijo'] = df_2['Tipo_Objeto_Fijo'].fillna('No aplica')
df_2['Tipo_Objeto_Fijo'].isnull().sum()

np.int64(0)

In [20]:
#Verificamos los cambios
df_2['Tipo_Objeto_Fijo'].unique()

array(['No aplica', 'Vehiculo', 'Tren', 'Arbol', 'Semaforo',
       'Objeto fijo', 'Poste', 'Inmueble', 'Muro', 'Semoviente',
       'Por identificar', 'Baranda', 'Vehiculo estacionado',
       'Valla-señal', 'Otro', 'Tarima-caseta', 'Hidrante'], dtype=object)

In [21]:
#Imputacion de datos faltantes en la columna elemento_choque con el valor "Sin informacion"
df_2['Elemento_Choque'] = df_2['Elemento_Choque'].fillna('Sin informacion')
df_2['Elemento_Choque'].isnull().sum()

np.int64(0)

In [22]:
#Verificamos los cambios
df_2['Elemento_Choque'].unique()

array(['Vehículo', 'Objeto Fijo', 'Sin informacion', 'Semoviente', 'Tren'],
      dtype=object)

In [23]:
#Normalizar los registros de la columna Localidad, validacion de cambios
df_2['Localidad'] = df_2['Localidad'].str.capitalize()
df_2['Localidad'].unique()

array(['Fontibón', 'Suba', 'Ciudad bolívar', 'San cristóbal', 'Bosa',
       'Kennedy', 'Usaquén', 'Chapinero', 'Engativá', 'Los mártires',
       'Barrios unidos', 'Puente aranda', 'Candelaria', 'Santa fe',
       'Teusaquillo', 'Tunjuelito', 'Antonio nariño',
       'Rafael uribe uribe', 'Usme', 'Sumapaz'], dtype=object)

In [24]:
#comprabamos los registros nulos
df_2.isnull().sum()

Codigo_Accidente                    0
Longitud                          288
Latitud                           288
Direccion                           0
Fecha_Accidente                     0
Año_Accidente                       0
Mes_Accidente                       0
Día_Accidente                       0
Día_Semana_Accidente                0
Hora_Accidente                      0
Minuto_Accidente                    0
Localidad                           0
Clase_Accidente                     0
Elemento_Choque                     0
Tipo_Objeto_Fijo                    0
Gravedad_Indicador_Tradicional      0
Gravedad_indicador_30d             11
dtype: int64

In [25]:
#Imputacion de datos faltantes en la columna Gravedad_indicador_30 con el valor "Sin informacion"
df_2['Gravedad_indicador_30d'] = df_2['Gravedad_indicador_30d'].fillna('Sin informacion')
df_2['Gravedad_indicador_30d'].isnull().sum()

np.int64(0)

In [26]:
#comprabamos los registros nulos
df_2.isnull().sum()

Codigo_Accidente                    0
Longitud                          288
Latitud                           288
Direccion                           0
Fecha_Accidente                     0
Año_Accidente                       0
Mes_Accidente                       0
Día_Accidente                       0
Día_Semana_Accidente                0
Hora_Accidente                      0
Minuto_Accidente                    0
Localidad                           0
Clase_Accidente                     0
Elemento_Choque                     0
Tipo_Objeto_Fijo                    0
Gravedad_Indicador_Tradicional      0
Gravedad_indicador_30d              0
dtype: int64

In [31]:
df_2.head()

,Codigo_Accidente,Longitud,Latitud,Direccion,Fecha_Accidente,Año_Accidente,Mes_Accidente,Día_Accidente,Día_Semana_Accidente,Hora_Accidente,Minuto_Accidente,Localidad,Clase_Accidente,Elemento_Choque,Tipo_Objeto_Fijo,Gravedad_Indicador_Tradicional,Gravedad_indicador_30d
169947,10509738,-74.125084,4.672291,KR 86-CL 24 02,2020-01-31,2020,Enero,31,viernes,10,41,Fontibón,Choque,Vehículo,No aplica,Solo Daños,Solo Daños
169948,10509739,-74.051785,4.732306,CL 150-KR 48 12,2020-01-31,2020,Enero,31,viernes,21,23,Suba,Choque,Vehículo,No aplica,Solo Daños,Solo Daños
169949,10509740,-74.125649,4.672324,CL 24-KR 86 02,2020-01-31,2020,Enero,31,viernes,10,20,Fontibón,Choque,Vehículo,No aplica,Solo Daños,Solo Daños
169950,10509741,-74.135092,4.542815,AV AVENIDA BOYACA-CL 15D S 02,2020-01-31,2020,Enero,31,viernes,12,18,Ciudad bolívar,Choque,Vehículo,No aplica,Solo Daños,Solo Daños
169951,10509742,-74.100031,4.561922,KR 4B-CL 36 S 53,2020-01-31,2020,Enero,31,viernes,1,12,San cristóbal,Choque,Objeto Fijo,Vehiculo,Solo Daños,Solo Daños


**Normalizacion de la columna direcciones, apoyo mediante la investigacion de un script que realice todos los reemplazos y garanticen un formato unificado**

In [32]:
# normalizacion de la columna direccion para posteriormente imputar los datos faltantes con la libreria geopy y asi obtener las coordenadas de latitud y longitud de los siniestros que no cuentan con esta informacion

#  Función de normalización de direcciones
def normalizar_direccion(addr):
    if pd.isna(addr):
        return None
        
    addr = str(addr).upper().strip()
    
    # PRIMERO: Manejar las abreviaturas de direcciones (S, E, N, O) que van después del número
    # Esto evita que "CL " se convierta en "CALLESTE"
    
    # Patrón para direcciones como "CL 24 E" -> "CALLE 24 ESTE"
    addr = re.sub(r'(\b(?:CL|CALLE|KR|CARRERA|TR|TRANSVERSAL|DG|DIAGONAL)\s+\d+[A-Z]*)\s+([SENO])\s', r'\1 \2 ', addr)
    
    # Ahora aplicar los reemplazos principales
    reemplazos = {
        'KR ': 'CARRERA ',
        'CRA ': 'CARRERA ',
        'CL ': 'CALLE ',
        'CLL ': 'CALLE ',
        'TR ': 'TRANSVERSAL ',
        'TV ': 'TRANSVERSAL ',
        'DG ': 'DIAGONAL ',
        'AC ': 'AVENIDA CARRERA ',
        'AV ': 'AVENIDA ',
        'AV.': 'AVENIDA ',
        'VIA ': 'CARRERA ',
        'KM ': 'KILOMETRO ',
        
        # Separadores
        '-': ' CON ',
        ' - ': ' CON ',
        '_': ' CON ',
    }
    
    # Aplicar reemplazos principales
    for key, value in reemplazos.items():
        addr = addr.replace(key, value)
    
    # LUEGO: Aplicar reemplazos de direcciones cardinales
    reemplazos_cardinales = {
        ' S ': ' SUR ',
        ' ES ': ' ESTE ',
        ' E ': ' ESTE ',
        ' N ': ' NORTE ',
        ' O ': ' OESTE ',
        ' SE ': ' SURESTE ',
        ' NE ': ' NORESTE ',
        ' SO ': ' SUROESTE ',
        ' NO ': ' NOROESTE ',
    }
    
    for key, value in reemplazos_cardinales.items():
        addr = addr.replace(key, value)
    
    # Manejar casos especiales de AVENIDA
    addr = re.sub(r'AVENIDA\s+AVENIDA', 'AVENIDA', addr)
    
    # Manejar números con letras (ej: 85-CL -> 85 CALLE)
    addr = re.sub(r'(\d+)-([A-Z])', r'\1 \2', addr)
    
    # Manejar números con BIS (ej: 4FBIS -> 4F BIS)
    addr = re.sub(r'(\d+[A-Z]?)BIS', r'\1 BIS', addr)
    
    # Manejar direcciones con formato "KR 85-CL 52"
    addr = re.sub(r'(\b[CARRERA|CALLE|TRANSVERSAL|DIAGONAL]+\s+\d+[A-Z]*)\s*-\s*(\b[CARRERA|CALLE|TRANSVERSAL|DIAGONAL]+\s+\d+[A-Z]*)', r'\1 CON \2', addr)
    
    # Normalizar números (ej: "02" -> "2")
    addr = re.sub(r'\b0(\d)\b', r'\1', addr)
    
    # Eliminar caracteres especiales
    addr = addr.replace('.', '')
    addr = addr.replace(',', '')
    
    # Quitar espacios extra
    addr = ' '.join(addr.split())
    
    # Agregar ciudad y país si no está presente
    if 'BOGOT' not in addr and 'COLOMBIA' not in addr:
        addr = addr + ', BOGOTÁ, COLOMBIA'
    
    return addr

# 3️ Aplicar normalización
print("Iniciando normalización de direcciones...")
df_2['direccion_normalizada'] = df_2['Direccion'].apply(normalizar_direccion)

# 4️ Mostrar resultados de normalización
print("✓ Normalización completada")
print(f"Total de direcciones normalizadas: {len(df_2)}")
print(f"Direcciones nulas en original: {df_2['Direccion'].isna().sum()}")
print(f"Direcciones nulas en normalizado: {df_2['direccion_normalizada'].isna().sum()}")

# 5️ Mostrar ejemplos CORREGIDOS
print("\nEjemplos de normalización CORREGIDOS:")
print("-" * 80)
ejemplos_corregidos = [
    "KR 86-CL 24 02",
    "KR 85-CL 52 S 2", 
    "KR 81-CL 1F S 64",
    "CL 047-KR 14 2",
    "AV AVENIDA DEL SUR-KR 62B 02"
]

for ejemplo in ejemplos_corregidos:
    normalizada = normalizar_direccion(ejemplo)
    print(f"Original: {ejemplo}")
    print(f"Normalizada: {normalizada}")
    print("-" * 80)

# También mostrar algunos del DataFrame
print("\nPrimeros 5 ejemplos del DataFrame:")
print("-" * 80)
for i in range(min(5, len(df_2))):
    original = df_2['Direccion'].iloc[i]
    normalizada = df_2['direccion_normalizada'].iloc[i]
    print(f"Original: {original}")
    print(f"Normalizada: {normalizada}")
    print("-" * 80)

# 6️ Guardar datos con direcciones normalizadas (OPCIONAL)
#df_2.to_csv("siniestros_direcciones_normalizadas.csv", index=False, encoding='utf-8')
#print("\n✓ Archivo guardado: siniestros_direcciones_normalizadas.csv")

Iniciando normalización de direcciones...
✓ Normalización completada
Total de direcciones normalizadas: 105094
Direcciones nulas en original: 0
Direcciones nulas en normalizado: 0

Ejemplos de normalización CORREGIDOS:
--------------------------------------------------------------------------------
Original: KR 86-CL 24 02
Normalizada: CARRERA 86 CON CALLE 24 2, BOGOTÁ, COLOMBIA
--------------------------------------------------------------------------------
Original: KR 85-CL 52 S 2
Normalizada: CARRERA 85 CON CALLE 52 SUR 2, BOGOTÁ, COLOMBIA
--------------------------------------------------------------------------------
Original: KR 81-CL 1F S 64
Normalizada: CARRERA 81 CON CALLE 1F SUR 64, BOGOTÁ, COLOMBIA
--------------------------------------------------------------------------------
Original: CL 047-KR 14 2
Normalizada: CALLE 047 CON CARRERA 14 2, BOGOTÁ, COLOMBIA
--------------------------------------------------------------------------------
Original: AV AVENIDA DEL SUR-KR 62B

In [40]:
#Validamos la creacion de una nueva columna con las direcciones normalizadas
df_2.head(10)

,Codigo_Accidente,Longitud,Latitud,Direccion,Fecha_Accidente,Año_Accidente,Mes_Accidente,Día_Accidente,Día_Semana_Accidente,Hora_Accidente,Minuto_Accidente,Localidad,Clase_Accidente,Elemento_Choque,Tipo_Objeto_Fijo,Gravedad_Indicador_Tradicional,Gravedad_indicador_30d,direccion_normalizada
169947,10509738,-74.125084,4.672291,KR 86-CL 24 02,2020-01-31,2020,Enero,31,viernes,10,41,Fontibón,Choque,Vehículo,No aplica,Solo Daños,Solo Daños,"CARRERA 86 CON CALLE 24 2, BOGOTÁ, COLOMBIA"
169948,10509739,-74.051785,4.732306,CL 150-KR 48 12,2020-01-31,2020,Enero,31,viernes,21,23,Suba,Choque,Vehículo,No aplica,Solo Daños,Solo Daños,"CALLE 150 CON CARRERA 48 12, BOGOTÁ, COLOMBIA"
169949,10509740,-74.125649,4.672324,CL 24-KR 86 02,2020-01-31,2020,Enero,31,viernes,10,20,Fontibón,Choque,Vehículo,No aplica,Solo Daños,Solo Daños,"CALLE 24 CON CARRERA 86 2, BOGOTÁ, COLOMBIA"
169950,10509741,-74.135092,4.542815,AV AVENIDA BOYACA-CL 15D S 02,2020-01-31,2020,Enero,31,viernes,12,18,Ciudad bolívar,Choque,Vehículo,No aplica,Solo Daños,Solo Daños,"AVENIDA BOYACA CON CALLE 15D SUR 2, BOGOTÁ, CO..."
169951,10509742,-74.100031,4.561922,KR 4B-CL 36 S 53,2020-01-31,2020,Enero,31,viernes,1,12,San cristóbal,Choque,Objeto Fijo,Vehiculo,Solo Daños,Solo Daños,"CARRERA 4B CON CALLE 36 SUR 53, BOGOTÁ, COLOMBIA"
169952,10509743,-74.096272,4.757223,KR 111-CL 153 30,2020-01-31,2020,Enero,31,viernes,13,44,Suba,Choque,Vehículo,No aplica,Solo Daños,Solo Daños,"CARRERA 111 CON CALLE 153 30, BOGOTÁ, COLOMBIA"
169953,10509744,-74.180517,4.598093,CL 65-KR 77H S 2,2020-01-31,2020,Enero,31,viernes,6,10,Bosa,Choque,Vehículo,No aplica,Solo Daños,Solo Daños,"CALLE 65 CON CARRERA 77H SUR 2, BOGOTÁ, COLOMBIA"
169954,10509745,-74.154019,4.642562,AV AVENIDA CIUDAD DE CALI-CL 6C 02,2020-01-31,2020,Enero,31,viernes,10,0,Kennedy,Choque,Vehículo,No aplica,Solo Daños,Solo Daños,"AVENIDA CIUDAD DE CALI CON CALLE 6C 2, BOGOTÁ,..."
169955,10509746,-74.123855,4.740633,CL 130A-KR 154 2,2020-01-31,2020,Enero,31,viernes,10,58,Suba,Choque,Vehículo,No aplica,Solo Daños,Solo Daños,"CALLE 130A CON CARRERA 154 2, BOGOTÁ, COLOMBIA"
169956,10509747,-74.028708,4.704716,KR 7-CL 127B 02,2020-01-31,2020,Enero,31,viernes,9,0,Usaquén,Choque,Vehículo,No aplica,Solo Daños,Solo Daños,"CARRERA 7 CON CALLE 127B 2, BOGOTÁ, COLOMBIA"


In [41]:
df_2.isnull().sum()

Codigo_Accidente                    0
Longitud                          288
Latitud                           288
Direccion                           0
Fecha_Accidente                     0
Año_Accidente                       0
Mes_Accidente                       0
Día_Accidente                       0
Día_Semana_Accidente                0
Hora_Accidente                      0
Minuto_Accidente                    0
Localidad                           0
Clase_Accidente                     0
Elemento_Choque                     0
Tipo_Objeto_Fijo                    0
Gravedad_Indicador_Tradicional      0
Gravedad_indicador_30d              0
direccion_normalizada               0
dtype: int64

**Imputacion de los registros nulos en las columnas latitud y logitud**

In [37]:
%pip install geopandas

from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import pandas as pd
import geopandas as gpd
import numpy as np
import os
import unicodedata

   ---------------------------------------- 0.0/19.2 MB ? eta -:--:--
   ------------------------- -------------- 12.3/19.2 MB 57.3 MB/s eta 0:00:01
   ---------------------------------------- 19.2/19.2 MB 52.7 MB/s  0:00:00
   ---------------------------------------- 0.0/6.3 MB ? eta -:--:--
   ---------------------------------------- 6.3/6.3 MB 51.3 MB/s  0:00:00
   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ---------------------------------------- 1.7/1.7 MB 40.0 MB/s  0:00:00

   ---------------------------------------- 0/4 [shapely]
   ---------------------------------------- 0/4 [shapely]
   ---------------------------------------- 0/4 [shapely]
   ---------- ----------------------------- 1/4 [pyproj]
   ---------- ----------------------------- 1/4 [pyproj]
   -------------------- ------------------- 2/4 [pyogrio]
   -------------------- ------------------- 2/4 [pyogrio]
   -------------------- ------------------- 2/4 [pyogrio]
   -------------------- --

In [ ]:
# Este proceso puede tardar entre 16 y 18 minutos dependiendo del número de filas y la velocidad de la API de geocodificación.
# Función para normalizar texto

def normalizar_texto(texto):
    if pd.isna(texto):
        return ""
    # Quitar tildes
    texto = ''.join(
        c for c in unicodedata.normalize('NFD', str(texto))
        if unicodedata.category(c) != 'Mn'
    )
    # Mayúscula sostenida y sin espacios extras
    return texto.strip().upper()


# 1. Inicializar geocodificador

geolocator = Nominatim(user_agent="geo_app")
geocode = RateLimiter(
    geolocator.geocode,
    min_delay_seconds=1,
    return_value_on_exception=None
)


# 2. Cargar datos de siniestros

# crear un un nuevo dataframe para no alterar el "original"
#df = df_2.copy()


# 3. Cargar localidades de Bogotá (GeoJSON)

url_localidades = "https://bogota-laburbano.opendatasoft.com/explore/dataset/poligonos-localidades/download/?format=geojson&lang=es"
gdf_localidades = gpd.read_file(url_localidades)
gdf_localidades["centroide"] = gdf_localidades.centroid

# Crear columna normalizada
gdf_localidades["LOC_NORMALIZADA"] = gdf_localidades["Nombre de la localidad"].apply(normalizar_texto)


# 4. Iterar y completar coordenadas

for i, row in df_2.iterrows():
    try:
        if pd.isna(row["Longitud"]) or pd.isna(row["Latitud"]):
            # 4.1 Intentar con Nominatim (Dirección)
            if "Direccion" in df_2.columns and pd.notna(row["direccion_normalizada"]):
                location = geocode(row["direccion_normalizada"])
                if location:
                    df_2.at[i, "Latitud"] = location.latitude
                    df_2.at[i, "Longitud"] = location.longitude
                    continue  # pasa a la siguiente fila

            # 4.2 Fallback: usar centroide de la localidad
            if "Localidad" in df_2.columns and pd.notna(row["Localidad"]):
                loc_df = normalizar_texto(row["Localidad"])
                match = gdf_localidades[gdf_localidades["LOC_NORMALIZADA"] == loc_df]

                if not match.empty:
                    centro = match.iloc[0]["centroide"]
                    df_2.at[i, "Latitud"] = centro.y
                    df_2.at[i, "Longitud"] = centro.x

    except Exception as e:
        print(f"Error en fila {i}: {e}")


# 5. Guardar resultado

df_2.to_excel("../data/processed/df_siniestros_limpio.xlsx", index=False)
print("Archivo guardado: df_siniestros_limpio.xlsx")

Skipping field geo_point_2d: unsupported OGR type: 3
C:\Users\andre\AppData\Local\Temp\ipykernel_91032\2184729943.py:36: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf_localidades["centroide"] = gdf_localidades.centroid
RateLimiter caught an error, retrying (0/2 tries). Called with (*('TRANSVERSAL 45 CON AVENIDA CARRERA 72 2, BOGOTÁ, COLOMBIA',), **{}).
Traceback (most recent call last):
  File "c:\Users\andre\OneDrive\Documentos\bootcamp analisis de datos\analisis de datos\ejercicio_11092025\venv\Lib\site-packages\urllib3\connectionpool.py", line 534, in _make_request
    response = conn.getresponse()
  File "c:\Users\andre\OneDrive\Documentos\bootcamp analisis de datos\analisis de datos\ejercicio_11092025\venv\Lib\site-packages\urllib3\connection.py", line 565, in getresponse
    httplib_response = super().getresponse()
  File "C:\Users\andre

Archivo guardado: df_siniestros_limpio.xlsx


In [45]:
#verificamos los 288 registros nulos hayan sido imputados y que el proceso de exportacion se haya realizado correctamente
df_2.isnull().sum()

Codigo_Accidente                  0
Longitud                          0
Latitud                           0
Direccion                         0
Fecha_Accidente                   0
Año_Accidente                     0
Mes_Accidente                     0
Día_Accidente                     0
Día_Semana_Accidente              0
Hora_Accidente                    0
Minuto_Accidente                  0
Localidad                         0
Clase_Accidente                   0
Elemento_Choque                   0
Tipo_Objeto_Fijo                  0
Gravedad_Indicador_Tradicional    0
Gravedad_indicador_30d            0
direccion_normalizada             0
dtype: int64

In [13]:
# Exportar los tres DataFrames en un solo archivo Excel
with pd.ExcelWriter("../data/processed/_Normalizacion_Base_Siniestros_.xlsx", engine="openpyxl") as writer:
    df_riesgo.to_excel(writer, sheet_name="Factores_Riesgo", index=False)
    df_transporte.to_excel(writer, sheet_name="Tipo_Transporte", index=False)
    df_2.to_excel(writer, sheet_name="_siniestros_Normalizada", index=False)

Exception ignored in: <function ZipFile.__del__ at 0x000001ED34BF9B20>
Traceback (most recent call last):
  File "C:\Users\andre\AppData\Local\Programs\Python\Python313\Lib\zipfile\__init__.py", line 1988, in __del__
    self.close()
  File "C:\Users\andre\AppData\Local\Programs\Python\Python313\Lib\zipfile\__init__.py", line 2005, in close
    self.fp.seek(self.start_dir)
ValueError: seek of closed file
